In [3]:
import pandas as pd
from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
import random
import os

ModuleNotFoundError: No module named 'tqdm'

In [16]:
train_df = pd.read_csv("../data/CelebA/train.txt", delimiter=" ", header=None)
train_df.columns = ["filename", "gender"]
test_df = pd.read_csv("../data/CelebA/test.txt", delimiter=" ", header=None)
test_df.columns = ["filename", "gender"]

df = pd.concat([train_df, test_df])
male_df = df[df["gender"] == 1]
female_df = df[df["gender"] == 0]

# randomly choose 500 female
male_filenames = random.sample(male_df["filename"].tolist(), 500)
assert len(male_filenames) == 500

# randomly choose 500 male
female_filenames = random.sample(female_df["filename"].tolist(), 500)
assert len(female_filenames) == 500

In [17]:
IMG_SIZE = (150, 150)

# remove the projection head from the pre-trained model and only keep the encoder
pretrained_model = keras.models.load_model("../output/supcon_pretraining/20221109-092421/supcon_model_600_6.95", compile=False)
inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
outputs = pretrained_model.layers[1](inputs)
encoder = keras.Model(inputs=inputs, outputs=outputs)
encoder.summary()

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2022-11-10 00:03:33.684159: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 model (Functional)          (None, 512)               11186119  
                                                                 
Total params: 11,186,119
Trainable params: 11,176,512
Non-trainable params: 9,607
_________________________________________________________________


In [26]:
male_vector_rep = []
for f in male_filenames:
    img = keras.utils.load_img(os.path.join("../data/CelebA/img_align_celeba", f))
    img = keras.utils.img_to_array(img)
    img = tf.image.resize(img, size=IMG_SIZE)

    male_vector_rep.append(encoder(tf.expand_dims(img, axis=0)))

KeyboardInterrupt: 